In [1]:
import os
from os.path import dirname

root_dir = dirname(os.getcwd())
os.chdir(root_dir)

In [2]:
import pickle
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import multiprocessing as mp

from sklearn.model_selection import StratifiedKFold
from src.nn import *
from src.utils import *

In [3]:
random_seed = [0, 42, 1234, 1337]

In [4]:
with open('data/vehicle_data.pkl', 'rb') as f:
    vehicle_data = pickle.load(f)

In [5]:
learning_rate = 0.001
num_epochs = 20
num_folds = 5

In [6]:
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)

In [7]:
nn_model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()

In [8]:
processes = []

for i, (train_index, test_index) in enumerate(skf.split(vehicle_data['data'], vehicle_data['label'])):
    train_subset = {
        'data': vehicle_data['data'][train_index],
        'label': vehicle_data['label'][train_index],
    }
    test_subset = {
        'data': vehicle_data['data'][test_index],
        'label': vehicle_data['label'][test_index],
    }

    train_dl = get_dataloader(train_subset)
    test_dl = get_dataloader(test_subset)

    for seed in random_seed:
        process = mp.Process(
            target=train_model,
            args=(
                'neural_network',
                nn_model,
                criterion,
                train_dl,
                test_dl,
                learning_rate,
                num_epochs,
                seed,
                i+1,
            )
        )

        processes.append(process)

In [ ]:
for process in processes:
    process.start()

for process in processes:
    process.join()

print('Training complete for all seeds.', flush=True)

Training completed for seed 42, fold 4
Training completed for seed 1337, fold 5
Training completed for seed 1337, fold 2
Training completed for seed 0, fold 2
Training completed for seed 0, fold 4
Training completed for seed 1337, fold 4
Training completed for seed 42, fold 1
Training completed for seed 42, fold 3
Training completed for seed 1234, fold 2
Training completed for seed 1234, fold 4
Training completed for seed 0, fold 1
Training completed for seed 1337, fold 1
Training completed for seed 42, fold 2
Training completed for seed 1234, fold 1
Training completed for seed 0, fold 3
Training completed for seed 1234, fold 3
Training completed for seed 1234, fold 5
Training completed for seed 1337, fold 3
Training completed for seed 0, fold 5
Training completed for seed 42, fold 5
Training complete for all seeds.
